<img src="logo_UCLA_blue_boxed.svg" alt="ucla logo" width="150" align="left"/>

### **MAE 150A - Intermediate Fluid Mechanics**

<hr />

# Vorticity, streamfunction, and velocity

In this notebook we will discuss the relationship between vorticity ($\omega$), velocity ($\mathbf{u}$), and streamfunction ($\psi$), in two-dimensional flows:

$$ \nabla^2\psi = -\omega $$

and

$$ \mathbf{u} = \nabla\times (\psi \mathbf{k}) $$

We will do this by considering some example vorticity distributions, formed from one or more vortices of the form

$$ \omega(x,y) = \dfrac{\Gamma}{\pi \sigma^2} \mathrm{e}^{-((x-x0)^2+(y-y0)^2)/\sigma^2} $$

This shape is called a *Gaussian*, and physically, represents a type of vortex called an Oseen vortex. The parameter $\Gamma$ is the circulation of this vortex, $\sigma$ is its radius, and $(x0,y0)$ its center.

### Set up the module

In [ ]:
using MAE150A

In [ ]:
pyplot()
initialize_environment()

### Set up a grid on which to construct our fields
You probably will not have any reason to change these, but you're welcome to play around, of course.

In [ ]:
# grid cell size
Δx = 0.02

# set the size of a region in which to set up the flow
xlim = (-2,2)
ylim = (-2,2)

# make the grid
g = PhysicalGrid(xlim,ylim,Δx)

Set up some other data structures for later use. You shouldn't need to touch these.

In [ ]:
ω = Nodes(Dual,size(g))
umag = Nodes(Primal,size(g))

xg,yg = coordinates(ω,g) # coordinates of grid points at which vorticity is evaluated
xc,yc = coordinates(umag,g) # coordinates of grid points at which velocity magnitude is evaluated

# Discrete Laplacian operator and its inverse
L = plan_laplacian(ω,with_inverse=true);

### Now set up a single vortex at the origin and let's look at it

In [ ]:
Γ = 1
σ = 0.2;
vort = OseenVortex(0,0,Γ,σ)

The next line evaluates the vortex at all points on the grid.

Note: We use a couple of Julia tricks here. It's not important to know the details. But if you're interested: the apostrophe after yg means `transpose`. So `xg`,`yg'` gives the coordinates of the rows and columns of the Cartesian grid. The `@.` means *vectorize* the next operation, i.e., perform this operation at every value of `xg` and every value of `yg` and assign the result to the array `ω`.

In [ ]:
@. ω = vort(xg,yg');

Now plot the vorticity field

In [ ]:
plot(xg,yg,ω,xlim=(-2,2),ylim=(-2,2),xlabel=L"x",ylabel=L"y",title="Vorticity field")

#### Now let's compute the streamfunction
We need to solve the Poisson equation to get the streamfunction, $\psi$:

$$ \nabla^2 \psi = -\omega $$

Details, if interested: We have set up the $L$ operator to be a discrete version of $\nabla^2$ on the grid (using finite differences). But strictly speaking, $L \approx \Delta x^2 \nabla^2$, where $\Delta x$ is the grid spacing. The inverse of $L$ is also available. We use the *backslash* (`\`) to provide this inverse. So the discrete solution is

In [ ]:
ψ = -Δx^2*(L\ω);

Plot the streamfunction contours

In [ ]:
plot(xg,yg,ψ,color=:black,xlim=(-2,2),ylim=(-2,2),xlabel=L"x",ylabel=L"y",title="Streamfunction")

Remember that each of these contours of streamfunction is a **streamline**. The only drawback is that the arrows aren't on it. (How would you determine the direction of the streamlines?)

It probably is not a surprise that the streamlines of a radially-symmetric vortex are a bunch of concentric circles. This makes it clear that a vortex is a region of rotational flow.

#### Finally, compute the velocity field
Here, we take the curl of the streamfunction (treating the streamfunction as a vector directed out of the screen),

$$ \mathbf{u} = \nabla\times (\psi\mathbf{k}) $$

The discrete curl operator is simply called `curl` but we have to divide by the grid spacing to get the correct approximation of the actual curl.

In [ ]:
u = curl(ψ)/Δx;

Now plot them

In [ ]:
xu, yu, xv, yv = coordinates(u,g)
ps1 = plot(xu,yu,u.u,xlim=(-2,2),ylim=(-2,2),xlabel=L"x",ylabel=L"y",title="u component",colorbar=:true)
ps2 = plot(xv,yv,u.v,xlim=(-2,2),ylim=(-2,2),xlabel=L"x",ylabel=L"y",title="v component",colorbar=:true)
plot(ps1,ps2,size=(900,450))

These are not as obvious to interpret. But with a little bit of thought, you can see that regions of large $x$ velocity are above and below the vortex, and regions of large $y$ velocity are on the sides.

#### The velocity magnitude

The speed of the flow, $|\mathbf{u}|$, is a little easier to interpret. But where is the speed the largest?

In [ ]:
umag = mag(u);
plot(xc,yc,umag,xlim=(-2,2),ylim=(-2,2),xlabel=L"x",ylabel=L"y",title="Velocity magnitude",colorbar=:true)

To see the speed distribution a little better, try plotting along a slice $y = 0$.

For those that like details: To get this slice, we construct a function that interpolates the velocity magnitude field on the grid, and we can then evaluate that function at any x,y location we like. Here, we evaluate it at $y = 0$ in a range of points along the $x$ axis.

In [ ]:
umag_fcn = interpolatable_field(xc,yc,umag);

In [ ]:
plot(xc,umag_fcn.(xc,0),xlim=(-2,2),ylim=(0,1),xlabel=L"x",ylabel=L"|\mathbf{u}(x,0)|",title="Velocity magnitude",legend=false)

The velocity is actually zero at the center and maximum a little bit away from the center. Try changing $\sigma$ and seeing how this location of maximum speed changes.

### Construct a pair of vortices

In [ ]:
Γ = 1
σ = 0.3
v1 = OseenVortex(-1,0,Γ,σ)
v2 = OseenVortex(1,0,Γ,σ)

In [ ]:
@. ω = v1(xg,yg') + v2(xg,yg');

In [ ]:
plot(xg,yg,ω,xlim=(-2,2),ylim=(-2,2),xlabel=L"x",ylabel=L"y",title="Vorticity field")

In [ ]:
ψ = -Δx^2*(L\ω);

In [ ]:
plot(xg,yg,ψ,color=:black,xlim=(-2,2),ylim=(-2,2),xlabel=L"x",ylabel=L"y",title="Streamfunction")

#### Stagnation points
Where are the stagnation points for this flow? There are a few ways to hunt for them:
* where streamlines cross
* where velocity components both vanish
* where velocity magnitude is zero

Let's try the second approach. We will plot the zero contour (the contour for which the value is zero) of the $x$ and $y$ components of velocity and lay them on top of each other. Stagnation points correspond to where the zero contours of each component cross. We will put the streamlines in the plot, too, just for reference.

In [ ]:
u = curl(ψ)/Δx;
xu, yu, xv, yv = coordinates(u,g)
plot(xg,yg,ψ,color=:lightgray)
plot!(xu,yu,u.u,levels=[0],color=:red,xlim=(-2,2),ylim=(-2,2)) # x component in red
plot!(xv,yv,u.v,levels=[0],color=:blue) # y component in red

So there are three stagnation points: one near each of the vortex centers and a third one right at the origin.

### Make some other vorticity fields and explore the associated streamlines here
Add more vortices, change their locations, their strengths (positive or negative), their radius. Explore!